# Friendnet - Classifying Friendly Faces

The objective of this project is to build models that are capable of detecting and classifying faces. We'd build a custom model and finetune on two other models - Resnet and Open AI's Clip model

# Import All Libraries

In [ ]:
import os
import pyheif
import imageio
import random
import shutil
import types
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import seaborn as sns
import open_clip
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import numpy as np
import cv2
import numpy as np
# !pip install pyheif
# !pip install grad-cam
# !pip install open-clip-torch

# Dataset Preprocessing

## Preprocess Setup

In [ ]:
# List friends and Tensor transformation
friends = ["tatyana", "uzair", "jacob", "luke", "osama"]
base_dir = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/"
transform = transforms.Compose([
    transforms.Resize((224, 224))
])

## Extract Frames from Videos

In [ ]:
standard_size = (1022, 1080)  # (width, height)

for friend in ["tatyana", "uzair", "jacob", "luke", "osama"]:
    print(f"\n Extracting for {friend}")
    friend_dir = os.path.join(base_dir, friend)
    video_out = os.path.join(friend_dir, "video_frames")
    os.makedirs(video_out, exist_ok=True)

    videos = ["video.mov"]
    if friend == "osama":
        videos.append("video2.mov")

    for vid in videos:
        video_path = os.path.join(friend_dir, vid)
        try:
            reader = imageio.get_reader(video_path)
            for idx, frame in tqdm(enumerate(reader), desc=f"{friend} - {vid}"):
                if idx % 10 == 0:
                    img = Image.fromarray(frame)

                    img = img.resize(standard_size)

                    out_name = f"{vid.replace('.MOV','')}_{idx:04d}.jpg"
                    img.save(os.path.join(video_out, out_name))
        except Exception as e:
            print(f"Error with {friend} - {vid}: {e}")


### Preprocess Extracted Frames

In [ ]:
for friend in ["tatyana", "uzair", "jacob", "luke", "osama"]:
    input_dir = os.path.join(base_dir, friend, "video_frames")
    output_dir = os.path.join(base_dir, "FRIENDNETFULL", friend)
    os.makedirs(output_dir, exist_ok=True)

    print(f"\n Preprocessing frames for {friend}")
    files = [f for f in os.listdir(input_dir) if f.endswith(".jpg")]
    for i, file in enumerate(tqdm(files, desc=f"{friend} - transform & save")):
        img = Image.open(os.path.join(input_dir, file)).convert("RGB")
        out_img = transform(img)
        out_img.save(os.path.join(output_dir, f"{i:04d}.jpg"))

## Split dataset 70% Training, 15% Test and 15% Validation

In [ ]:
# Define source and output paths
source_dir = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETFULL"
split_dir = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETSET"

friends = ["tatyana", "uzair", "jacob", "luke", "osama"]
target_counts = {"train": 700, "val": 150, "test": 150}

# Create output folders
for split in target_counts:
    for friend in friends:
        os.makedirs(os.path.join(split_dir, split, friend), exist_ok=True)

# Split and copy images
for friend in tqdm(friends, desc="Splitting friends"):
    src = os.path.join(source_dir, friend)
    images = [f for f in os.listdir(src) if f.endswith(".jpg")]
    random.shuffle(images)

    # Determine base sizes first
    train_cut = min(target_counts["train"], int(0.7 * len(images)))
    val_cut = min(target_counts["val"], int(0.15 * len(images)))
    test_cut = min(target_counts["test"], len(images) - train_cut - val_cut)

    train_imgs = images[:train_cut]
    val_imgs = images[train_cut:train_cut + val_cut]
    test_imgs = images[train_cut + val_cut:train_cut + val_cut + test_cut]

    for split_name, split_imgs in zip(["train", "val", "test"], [train_imgs, val_imgs, test_imgs]):
        for img in split_imgs:
            src_path = os.path.join(src, img)
            dst_path = os.path.join(split_dir, split_name, friend, img)
            shutil.copy(src_path, dst_path)

## Augment Missing Images

In [ ]:
# Define transformations and augmentation
augment_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    T.RandomResizedCrop(224, scale=(0.9, 1.0), ratio=(0.9, 1.1))
])

def augment_images(split, friend, target_count):
    input_dir = os.path.join(split_dir, split, friend)
    images = [f for f in os.listdir(input_dir) if f.endswith(".jpg")]
    current_count = len(images)

    if current_count >= target_count:
        print(f"{friend} - {split}: already has {current_count}")
        return

    needed = target_count - current_count
    print(f"Augmenting {friend} - {split}: {current_count} → {target_count} (+{needed})")

    for i in tqdm(range(needed), desc=f"{friend}-{split}", leave=False):
        original = random.choice(images)
        img_path = os.path.join(input_dir, original)

        try:
            img = Image.open(img_path).convert("RGB")
            aug_img = augment_transform(img)
            aug_name = f"aug_{i:04d}.jpg"
            aug_img.save(os.path.join(input_dir, aug_name))
        except Exception as e:
            print(f"Error on {img_path}: {e}")

### Augment Across Splits

In [ ]:
for split, count in target_counts.items():
    print(f"\n Split: {split.upper()}")
    for friend in tqdm(friends, desc=f"Augmenting {split}", leave=True):
        augment_images(split, friend, count)

print("\n All splits and classes now balanced: 700 train / 150 val / 150 test each.")

## Preview Before/After (One per Friend)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

for friend in friends:
    print(f"\n Previewing {friend}")
    raw_dir = os.path.join(base_dir, friend, "video_frames")

    raw_images = [f for f in os.listdir(raw_dir) if f.endswith(".jpg")]
    if not raw_images:
        print(f"No image found for {friend}")
        continue

    raw_path = os.path.join(raw_dir, raw_images[0])
    img = Image.open(raw_path).convert("RGB")
    img_trans = transform(img)  # Already includes ToTensor()

    plt.figure(figsize=(8, 4))
    plt.suptitle(f"{friend.capitalize()} — Original vs Transformed", fontsize=16)

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(img_trans.permute(1, 2, 0))  # Convert CxHxW to HxWxC for display
    plt.title("Transformed")
    plt.axis("off")

    plt.show()

# Configurations

In [ ]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set Seeders for Reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset Loaders
base_path = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETSET"

train_dataset = datasets.ImageFolder(root=os.path.join(base_path, "train"), transform=transform)
val_dataset   = datasets.ImageFolder(root=os.path.join(base_path, "val"), transform=transform)
test_dataset  = datasets.ImageFolder(root=os.path.join(base_path, "test"), transform=transform)

batch_size = 30

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)

## Check Visuals for a Batch

In [ ]:
images, labels = next(iter(train_loader))
grid_img = torchvision.utils.make_grid(images[:8], nrow=4)
plt.figure(figsize=(10,5))
plt.imshow(grid_img.permute(1, 2, 0))
plt.title([train_dataset.classes[label] for label in labels[:8]])
plt.axis('off')
plt.show()

# Build Custom CNN (FriendNet)

In [ ]:
class FriendNet(nn.Module):
  def __init__(self, num_classes=5):
    super(FriendNet, self).__init__()

    # Convolutional Layers
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2)

    # Batch Norms & Dropouts
    self.bn1 = nn.BatchNorm2d(64)
    self.bn2 = nn.BatchNorm2d(128)
    self.bn3 = nn.BatchNorm2d(256)

    self.dropout = nn.Dropout(0.5)

    # Dense Layer
    self.fc = nn.Linear(256 * 28 * 28, 512)
    self.out = nn.Linear(512, num_classes)

  def forward(self, x):
    x = self.pool(F.relu(self.bn1(self.conv1(x))))
    x = self.pool(F.relu(self.bn2(self.conv2(x))))
    x = self.pool(F.relu(self.bn3(self.conv3(x))))

    x = x.view(x.size(0), -1)
    x = self.dropout(F.relu(self.fc(x)))
    x = self.out(x)
    return x

print(FriendNet())

## Define Hyperparameters

In [ ]:
criterion = nn.CrossEntropyLoss()
model = FriendNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
epochs = 30

print("Loss Fn", criterion)
print("Batch Size", batch_size)
print("Epochs", epochs)
print("Optimizer", optimizer)

## Training & Validation

In [ ]:
print("Training...")

train_losses = []
train_accuracy = []
val_losses = []
val_accuracy = []

# EARLY STOPPING CONFIG
patience = 10  # Number of epochs to wait before stopping
best_val_loss = float('inf')
early_stop_counter = 0

# path to save the best model
best_model_path = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_friendnet_model.pth"

for epoch in tqdm(range(epochs), desc="Epochs"):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc="Train Loader"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        train_loss += loss.item() * images.size(0)

    train_losses.append(train_loss / total)
    train_accuracy.append(100 * train_correct / total)

    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation Loader"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, pred = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_total += labels.size(0)
            val_running_loss += loss.item() * images.size(0)
            val_correct += (pred == labels).sum().item()

    current_val_loss = val_running_loss / val_total
    current_val_accuracy = 100 * val_correct / val_total
    val_losses.append(current_val_loss)
    val_accuracy.append(current_val_accuracy)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_losses[-1]:.4f}, "
          f"Val Loss: {current_val_loss:.4f}, Val Acc: {current_val_accuracy:.2f}%")

    # EARLY STOPPING CHECK
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), best_model_path)
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Finished Training.")

### Train & Validation Visuals

In [ ]:
# Plot training loss over time
plt.figure()
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('FriendNet Training and Validation Loss')
plt.xlabel('mini-batches')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy over time
plt.figure()
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('FriendNet Training and Validation Accuracy')
plt.xlabel('mini-batches')
plt.ylabel('Accuracy (%)')
plt.legend()

### Feature Importance

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_friendnet_model.pth"))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

def preprocess(img_path):
    img = Image.open(img_path).convert("RGB").resize((224, 224))
    img_np = np.array(img).astype(np.float32) / 255.0
    tensor = normalize(transform(img)).unsqueeze(0).to(device)
    return img_np, tensor

target_layer = model.conv3

class_names = ["tatyana", "luke", "uzair", "jacob", "osama"]

cam = GradCAM(model=model, target_layers=[target_layer])

for class_idx, class_name in enumerate(class_names):
    folder = f"/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETSET/test/{class_name}"
    sample_image = random.choice(os.listdir(folder))

    img_path = os.path.join(folder, sample_image)
    original_img, input_tensor = preprocess(img_path)

    targets = [ClassifierOutputTarget(class_idx)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]

    visualization = show_cam_on_image(original_img, grayscale_cam, use_rgb=True)

    plt.imshow(visualization)
    plt.title(f"FriendNet Feature-Importance for {class_name.capitalize()}")
    plt.axis("off")
    plt.show()

## Model Evaluation

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"FriendNet Accuracy : {accuracy:.4f}")
print(f"FriendNet Precision: {precision:.4f}")
print(f"FriendNet Recall   : {recall:.4f}")
print(f"FriendNet F1 Score : {f1:.4f}")

### Evaluation Visuals

In [ ]:
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, cmap='viridis')
plt.title('FriendNet Classification Report')
plt.show()

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('FriendNet Confusion Matrix')
plt.show()

# Finetune On Resnet50

## Model Definition

In [ ]:
resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 5)

print(resnet)

## Define Hyperparameters

In [ ]:
criterion = nn.CrossEntropyLoss()
resnet = resnet.to(device)
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
epochs = 30

print("Loss Fn", criterion)
print("Batch Size", batch_size)
print("Epochs", epochs)
print("Optimizer", optimizer)

total_params = sum(p.numel() for p in resnet.parameters())
trainable_params = sum(p.numel() for p in resnet.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params} / {total_params}")

print(train_dataset.class_to_idx)

## Training and Validation

In [ ]:
print("Training...")

res_train_losses = []
res_train_accuracy = []
res_val_losses = []
res_val_accuracy = []

# EARLY STOPPING CONFIG
res_patience = 10  # Number of epochs to wait before stopping
res_best_val_loss = float('inf')
res_early_stop_counter = 0

# path to save the best model
res_best_model_path = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_resnet_model.pth"

for epoch in tqdm(range(epochs), desc="Epochs"):
    resnet.train()
    res_train_loss = 0.0
    res_train_correct = 0
    res_total = 0

    for images, labels in tqdm(train_loader, desc="Train Loader"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        res_total += labels.size(0)
        res_train_correct += (predicted == labels).sum().item()
        res_train_loss += loss.item() * images.size(0)

    res_train_losses.append(res_train_loss / res_total)
    res_train_accuracy.append(100 * res_train_correct / res_total)

    resnet.eval()
    res_val_running_loss = 0.0
    res_val_correct = 0
    res_val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation Loader"):
            images, labels = images.to(device), labels.to(device)

            outputs = resnet(images)
            _, pred = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            res_val_total += labels.size(0)
            res_val_running_loss += loss.item() * images.size(0)
            res_val_correct += (pred == labels).sum().item()

    res_current_val_loss = res_val_running_loss / res_val_total
    res_current_val_accuracy = 100 * res_val_correct / res_val_total
    res_val_losses.append(res_current_val_loss)
    res_val_accuracy.append(res_current_val_accuracy)

    print(f"Train Loss: {res_train_losses[-1]:.4f}, "
          f"Val Loss: {res_current_val_loss:.4f}, Val Acc: {res_current_val_accuracy:.2f}%")

    # EARLY STOPPING CHECK
    if res_current_val_loss < res_best_val_loss:
        res_best_val_loss = res_current_val_loss
        res_early_stop_counter = 0
        torch.save(resnet.state_dict(), res_best_model_path)
    else:
        res_early_stop_counter += 1
        if res_early_stop_counter >= res_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Finished Training.")

### Train and Validation Visuals

In [ ]:
# Plot training loss over time
plt.figure()
plt.plot(res_train_losses, label='Training Loss')
plt.plot(res_val_losses, label='Validation Loss')
plt.title('ResNet50 Training and Validation Loss')
plt.xlabel('mini-batches')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy over time
plt.figure()
plt.plot(res_train_accuracy, label='Training Accuracy')
plt.plot(res_val_accuracy, label='Validation Accuracy')
plt.title('ResNet50 Training and Validation Accuracy')
plt.xlabel('mini-batches')
plt.ylabel('Accuracy (%)')
plt.legend()

### Feature Importance

In [ ]:
resnet.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_resnet_model.pth"))
resnet.to(device)
resnet.eval()

# Define transforms for model input
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

def preprocess(img_path):
    # Load and resize image
    img = Image.open(img_path).convert("RGB").resize((224, 224))

    # For visualization (show_cam_on_image expects numpy array in [0,1] range)
    img_np = np.array(img).astype(np.float32) / 255.0

    # For model input (apply full transform pipeline)
    tensor = transform(img).unsqueeze(0).to(device)

    return img_np, tensor

target_layer = resnet.layer4

class_names = ["tatyana", "luke", "uzair", "jacob", "osama"]

cam = GradCAM(model=resnet, target_layers=[target_layer])

for class_idx, class_name in enumerate(class_names):
    folder = f"/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETSET/test/{class_name}"
    sample_image = random.choice(os.listdir(folder))

    img_path = os.path.join(folder, sample_image)
    original_img, input_tensor = preprocess(img_path)

    targets = [ClassifierOutputTarget(class_idx)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]

    visualization = show_cam_on_image(original_img, grayscale_cam, use_rgb=True)

    plt.imshow(visualization)
    plt.title(f"ResNet50 Feature-Importance for {class_name.capitalize()}")
    plt.axis("off")
    plt.show()

## Model Evaluation

In [ ]:
resnet.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet(inputs)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"ResNet50 Accuracy : {accuracy:.4f}")
print(f"ResNet50 Precision: {precision:.4f}")
print(f"ResNet50 Recall   : {recall:.4f}")
print(f"ResNet50 F1 Score : {f1:.4f}")

### Evaluation Visuals

In [ ]:
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, cmap='viridis')
plt.title('ResNet Classification Report')
plt.show()

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('ResNet Confusion Matrix')
plt.show()

# Finetune on CLIP Model

## Dataset Configurations

In [ ]:
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32', pretrained='openai'
)

print(clip_preprocess)

base_path = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/imageset/FRIENDNETSET"

train_dataset = datasets.ImageFolder(root=os.path.join(base_path, "train"), transform=clip_preprocess)
val_dataset   = datasets.ImageFolder(root=os.path.join(base_path, "val"),   transform=clip_preprocess)
test_dataset  = datasets.ImageFolder(root=os.path.join(base_path, "test"),  transform=clip_preprocess)

batch_size = 30

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)

## Model Definition

In [ ]:
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        embed_dim = clip_model.visual.output_dim
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, images, enable_grad=False):
        if enable_grad:
            # Allow gradients for Grad-CAM
            features = self.clip.encode_image(images)
        else:
            # Normal inference without gradients
            with torch.no_grad():
                features = self.clip.encode_image(images)
        return self.classifier(features)

num_classes = 5
clip = CLIPClassifier(clip_model, num_classes)

# UnFreeze the CLIP backbone, train only the classifier
for param in clip.parameters():
    param.requires_grad = True

print(clip)

## Define Hyperparameters

In [ ]:
criterion = nn.CrossEntropyLoss()
clip = clip.to(device)
optimizer = torch.optim.SGD(clip.parameters(), lr=0.0001, momentum=0.9)
epochs = 30

print("Loss Fn", criterion)
print("Batch Size", batch_size)
print("Epochs", epochs)
print("Optimizer", optimizer)

total_params = sum(p.numel() for p in clip.parameters())
trainable_params = sum(p.numel() for p in clip.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params} / {total_params}")

print(train_dataset.class_to_idx)

## Training and Validation

In [ ]:
print("Training...")

clip_train_losses = []
clip_train_accuracy = []
clip_val_losses = []
clip_val_accuracy = []

# EARLY STOPPING CONFIG
clip_patience = 10  # Number of epochs to wait before stopping
clip_best_val_loss = float('inf')
clip_early_stop_counter = 0

# path to save the best model
clip_best_model_path = "/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_clip_model.pth"

for epoch in tqdm(range(epochs), desc="Epochs"):
    clip.train()
    clip_train_loss = 0.0
    clip_train_correct = 0
    clip_total = 0

    for images, labels in tqdm(train_loader, desc="Train Loader"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = clip(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        clip_total += labels.size(0)
        clip_train_correct += (predicted == labels).sum().item()
        clip_train_loss += loss.item() * images.size(0)

    clip_train_losses.append(clip_train_loss / clip_total)
    clip_train_accuracy.append(100 * clip_train_correct / clip_total)

    clip.eval()
    clip_val_running_loss = 0.0
    clip_val_correct = 0
    clip_val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation Loader"):
            images, labels = images.to(device), labels.to(device)

            outputs = clip(images)
            _, pred = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            clip_val_total += labels.size(0)
            clip_val_running_loss += loss.item() * images.size(0)
            clip_val_correct += (pred == labels).sum().item()

    clip_current_val_loss = clip_val_running_loss / clip_val_total
    clip_current_val_accuracy = 100 * clip_val_correct / clip_val_total
    clip_val_losses.append(clip_current_val_loss)
    clip_val_accuracy.append(clip_current_val_accuracy)

    print(f"Train Loss: {clip_train_losses[-1]:.4f}, "
          f"Val Loss: {clip_current_val_loss:.4f}, Val Acc: {clip_current_val_accuracy:.2f}%")

    # EARLY STOPPING CHECK
    if clip_current_val_loss < clip_best_val_loss:
        clip_best_val_loss = clip_current_val_loss
        clip_early_stop_counter = 0
        torch.save(clip.state_dict(), clip_best_model_path)
    else:
        clip_early_stop_counter += 1
        if clip_early_stop_counter >= clip_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Finished Training.")

### Train Visuals

In [ ]:
# Plot training loss over time
plt.figure()
plt.plot(clip_train_losses, label='Training Loss')
plt.plot(clip_val_losses, label='Validation Loss')
plt.title('CLIP Training and Validation Loss')
plt.xlabel('mini-batches')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy over time
plt.figure()
plt.plot(clip_train_accuracy, label='Training Accuracy')
plt.plot(clip_val_accuracy, label='Validation Accuracy')
plt.title('CLIP Training and Validation Accuracy')
plt.xlabel('mini-batches')
plt.ylabel('Accuracy (%)')
plt.legend()

## Model Evaluation

In [ ]:
clip.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/Pytorch/Project/model/best_clip_model.pth"))
clip.to(device)
clip.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = clip(inputs)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"CLIP Accuracy : {accuracy:.4f}")
print(f"CLIP Precision: {precision:.4f}")
print(f"CLIP Recall   : {recall:.4f}")
print(f"CLIP F1 Score : {f1:.4f}")

### Evaluation Visuals

In [ ]:
class_names = ["tatyana", "luke", "uzair", "jacob", "osama"]

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, cmap='viridis')
plt.title('CLIP Classification Report')
plt.show()

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('CLIP Confusion Matrix')
plt.show()